# 1.MORDM OPTIMIZATION

# 1.1 Generate random policies

In [1]:
import pandas as pd
import numpy as np

from ema_workbench import load_results
from ema_workbench import Scenario

from ema_workbench import (Model, MultiprocessingEvaluator, Policy, Scenario)
from ema_workbench.em_framework.evaluators import perform_experiments
from ema_workbench.em_framework.samplers import sample_uncertainties
from ema_workbench.util import ema_logging
import time
from problem_formulation import get_model_for_problem_formulation
from ema_workbench.em_framework.optimization import (HyperVolume, EpsilonProgress)
from ema_workbench import Constraint
from ema_workbench import (Model, CategoricalParameter,
                           ScalarOutcome, IntegerParameter, RealParameter)

In [7]:
from ema_workbench import (Model, MultiprocessingEvaluator, Policy, Scenario, SequentialEvaluator)

from ema_workbench.em_framework.evaluators import perform_experiments
from ema_workbench.em_framework.samplers import sample_uncertainties
from ema_workbench.util import ema_logging

ema_logging.log_to_stderr(ema_logging.INFO)

<Logger EMA (DEBUG)>

In [3]:
from ema_workbench import load_results

results_deep = []
convergence_all = []

for i in range(5):
    result, convergence = load_results(f'../results/mordm_7500_rp_scenario{i}.tar.gz')
    results_deep.append(result)
    convergence_all.append(convergence)
    
    


[MainProcess/INFO] results loaded succesfully from D:\Willy\Documents\GitHub\Model_based_decision_making\EPA1361\final assignment\results\mordm_7500_rp_scenario0.tar.gz
[MainProcess/INFO] results loaded succesfully from D:\Willy\Documents\GitHub\Model_based_decision_making\EPA1361\final assignment\results\mordm_7500_rp_scenario1.tar.gz
[MainProcess/INFO] results loaded succesfully from D:\Willy\Documents\GitHub\Model_based_decision_making\EPA1361\final assignment\results\mordm_7500_rp_scenario2.tar.gz
[MainProcess/INFO] results loaded succesfully from D:\Willy\Documents\GitHub\Model_based_decision_making\EPA1361\final assignment\results\mordm_7500_rp_scenario3.tar.gz
[MainProcess/INFO] results loaded succesfully from D:\Willy\Documents\GitHub\Model_based_decision_making\EPA1361\final assignment\results\mordm_7500_rp_scenario4.tar.gz


0
1
2
3
4


In [18]:
for i, result in enumerate(results_deep):
    outcomes_of_interest = result.iloc[:, :-3]
outcomes_of_interest

,0_RfR 0,0_RfR 1,0_RfR 2,1_RfR 0,1_RfR 1,1_RfR 2,2_RfR 0,2_RfR 1,2_RfR 2,3_RfR 0,...,A.2_DikeIncrease 2,A.3_DikeIncrease 0,A.3_DikeIncrease 1,A.3_DikeIncrease 2,A.4_DikeIncrease 0,A.4_DikeIncrease 1,A.4_DikeIncrease 2,A.5_DikeIncrease 0,A.5_DikeIncrease 1,A.5_DikeIncrease 2
0,1,0,1,0,1,0,1,1,1,1,...,0,3,0,0,3,0,0,4,0,0
1,0,0,0,1,1,0,1,1,1,1,...,0,3,0,0,2,0,0,3,0,0
2,1,0,1,0,1,1,1,1,1,1,...,0,3,0,0,3,0,0,3,0,0
3,1,0,1,0,0,0,1,1,1,1,...,0,3,0,0,0,0,0,2,0,0
4,1,0,1,1,1,1,1,1,1,1,...,0,3,0,0,1,0,0,3,0,0
5,0,0,0,0,1,1,1,1,1,1,...,0,3,0,0,2,0,0,3,0,0
6,1,0,1,0,0,0,1,1,1,1,...,0,3,0,0,0,0,0,2,0,0
7,1,0,1,1,1,0,1,1,1,1,...,0,3,0,0,1,0,0,3,0,0


Re-evaluation

In [9]:
from ema_workbench import Policy

policies = []
for i, result in enumerate(results_deep):
    outcomes_of_interest = result.iloc[:, :-3]
    for j, row in outcomes_of_interest.iterrows():
        policy = Policy(f'scenario {i} policy {j}', **row.to_dict())
        policies.append(policy)

In [12]:
policies[1]

Policy({'0_RfR 0': 1, '0_RfR 1': 0, '0_RfR 2': 0, '1_RfR 0': 1, '1_RfR 1': 1, '1_RfR 2': 1, '2_RfR 0': 1, '2_RfR 1': 1, '2_RfR 2': 1, '3_RfR 0': 1, '3_RfR 1': 1, '3_RfR 2': 1, '4_RfR 0': 1, '4_RfR 1': 1, '4_RfR 2': 1, 'EWS_DaysToThreat': 3, 'A.1_DikeIncrease 0': 0, 'A.1_DikeIncrease 1': 0, 'A.1_DikeIncrease 2': 0, 'A.2_DikeIncrease 0': 4, 'A.2_DikeIncrease 1': 0, 'A.2_DikeIncrease 2': 0, 'A.3_DikeIncrease 0': 1, 'A.3_DikeIncrease 1': 0, 'A.3_DikeIncrease 2': 0, 'A.4_DikeIncrease 0': 1, 'A.4_DikeIncrease 1': 0, 'A.4_DikeIncrease 2': 0, 'A.5_DikeIncrease 0': 8, 'A.5_DikeIncrease 1': 0})

In [5]:
from problem_formulation import get_model_for_problem_formulation

# We use the original problem formulation to account for the RfR policies
dike_model, planning_steps = get_model_for_problem_formulation(2)

In [8]:
ema_logging.log_to_stderr(ema_logging.INFO)

with SequentialEvaluator(dike_model) as evaluator:
    reeevaluation_results = evaluator.perform_experiments(1000, policies=policies)
    
experiments_reev, outcomes_reev = reeevaluation_results

[MainProcess/INFO] performing 1000 scenarios * 42 policies * 1 model(s) = 42000 experiments
[MainProcess/INFO] performing experiments sequentially
[MainProcess/ERROR] 'DikeIncrease 2'
Traceback (most recent call last):
  File "C:\Users\Willy\Anaconda3\lib\site-packages\ema_workbench\em_framework\experiment_runner.py", line 85, in run_experiment
    model.run_model(scenario, policy)
  File "C:\Users\Willy\Anaconda3\lib\site-packages\ema_workbench\util\ema_logging.py", line 158, in wrapper
    res = func(*args, **kwargs)
  File "C:\Users\Willy\Anaconda3\lib\site-packages\ema_workbench\em_framework\model.py", line 338, in run_model
    outputs = self.run_experiment(experiment)
  File "C:\Users\Willy\Anaconda3\lib\site-packages\ema_workbench\util\ema_logging.py", line 158, in wrapper
    res = func(*args, **kwargs)
  File "C:\Users\Willy\Anaconda3\lib\site-packages\ema_workbench\em_framework\model.py", line 391, in run_experiment
    model_output = self.function(**experiment)
  File "D:\Wi

EMAError: exception in run_model
Caused by: KeyError: 'DikeIncrease 2'

In [ ]:
from ema_workbench import save_results
save_results((experiments_reev, outcomes_reev), f'../results/mordm_exp_2_1000.tar.gz')

# experiments_reev, outcomes_reev = load_results('../results/mordm_exp_1000.tar.gz') 
# outcomes_reev_df = pd.DataFrame.from_dict(outcomes_reev)

Comparing both cells below we can see how we have 66 unique policies.

In [ ]:
experiments_reev.iloc[:,19:-3].duplicated().value_counts()

In [ ]:
experiments_reev['policy'].nunique()

We now only chose the policies that keep the number of deaths under the threshold in ALL scenarios

In [ ]:
outcomes_reev_df.describe()

In [ ]:
outcomes_reev_df['policy']= experiments_reev['policy']
policy_names = outcomes_reev_df.policy.unique()

policies_of_interest = []
for policy in policy_names:
    policy_df = outcomes_reev_df[outcomes_reev_df['policy'] == policy]
    policy_df_logical = policy_df['Expected Number of Deaths'] > (0.0001*3*5)
    type(policy_df_logical)
    if policy_df_logical.any():
        continue
    else:
        policies_of_interest.append(policy)

policies_of_interest

We can see that there is no policy that obeys the threshold under all scenarios. We therefore calculate the robustness using the thresholds.......

In [ ]:
outcomes_reev_df_death= outcomes_reev_df[outcomes_reev_df['Expected Number of Deaths']<=(0.0001*3*5)]
outcomes_reev_df_death

In [ ]:
#Por ahora probamos sin evacuation costs y damage, que no estan especificados

max_deaths = 0.0001
max_rfr_costs = 1.1e9
max_dike_costs = 3.04e8

thresholds = {'Dike Investment Costs':max_dike_costs, 'RfR Investment Costs':max_rfr_costs, 'Expected Number of Deaths':max_deaths}

overall_scores = {}
for policy in experiments_reev.policy.unique():
    logical = experiments_reev.policy == policy
    scores = {}
    for k, v in outcomes_reev.items():
        try:
            n = np.sum(v[logical]<=thresholds[k])
        except KeyError:
            continue
        scores[k] = n/1000 
    overall_scores[policy] = scores
        
overall_scores = pd.DataFrame(overall_scores).T

In [ ]:
#overall_scores_filtered = overall_scores[(overall_scores['Expected Number of Deaths'] >= 0.990) & (overall_scores['RfR Investment Costs'] == 0.0)]

In [ ]:
limits = parcoords.get_limits(overall_scores)
paraxes = parcoords.ParallelAxes(limits)
paraxes.plot(overall_scores)
plt.show()

In [ ]:
import functools

def robustness(direction, threshold, data):
    if direction == 'SMALLER':
        return np.sum(data<=threshold)/data.shape[0]
    else:
        return np.sum(data>=threshold)/data.shape[0]

def costs(data):
    return data[0]/1e3 # makes numbers nicer CAREFUL WITH UNITS
    
SMALLER = 'SMALLER'


max_deaths = 0.0001
max_rfr_costs = 1.1e9
max_dike_costs = 3.04e8

Expected_Annual_Damage = costs
Dike_Investment_Costs = functools.partial(robustness, SMALLER, max_dike_costs)
Expected_Number_of_Deaths = functools.partial(robustness, SMALLER, max_deaths)
RfR_Investment_Costs = functools.partial(robustness, SMALLER, max_rfr_costs)
Evacuation_Costs = costs

In [ ]:
from ema_workbench.analysis import parcoords


funcs = {'Expected Annual Damage': Expected_Annual_Damage,
         'Dike Investment Costs': Dike_Investment_Costs,
         'Expected Number of Deaths':Expected_Number_of_Deaths,
         'RfR Investment Costs':RfR_Investment_Costs,       
         'Evacuation Costs': Evacuation_Costs}

total_scores = {}
for policy in np.unique(experiments_reev['policy']):
    scores = {}
    logical = experiments_reev['policy'] == policy
    
    temp_outcomes = {k:v[logical] for k,v in outcomes_reev.items()}
    
    for k, v in temp_outcomes.items():
        score = funcs[k](v)
        scores[k] = score
    total_scores[policy] = scores

data = pd.DataFrame(total_scores).T#.reset_index(drop=True)
limits = parcoords.get_limits(data)
# limits.loc[0, :] = 0
# limits.loc[1, :] = 1

paraxes = parcoords.ParallelAxes(limits)
paraxes.plot(data)
plt.show()

In [ ]:
data_death = data.sort_values(by=['Expected Number of Deaths']).tail(5)
data_death

In [ ]:
outcomes_names = list(outcomes_reev.keys())
n_policies = experiments_reev['policy'].nunique()
n_experiments = len(experiments_reev)
n_exp_pol = n_experiments/n_policies

In [ ]:
# To calculate the regret we find the maximum value for each outcome under each policy with the same values for the uncertainties
# Then, we calculate the regret and find the maximum regret to study which is the policy that has the lowest maximum regret
final_regret = {}
for outcome in outcomes_names:
    policy_dict = {}

    for i in range(1, n_policies+1):
        # We first find the maximum value for each outcome under all policies
        policy_row = outcomes_reev[outcome][int(n_exp_pol*(i-1)): int(n_exp_pol*i)]
        policy_dict['policy'+f"{i}"] = policy_row
    policy_df = pd.DataFrame.from_dict(policy_dict)
    maxValuesObj = policy_df.max(axis=1)
    regret_row = abs(policy_df.sub(maxValuesObj, axis='index'))
    final_regret[outcome] = regret_row.max()

In [ ]:
regret_df = pd.DataFrame.from_dict(final_regret)
limits = parcoords.get_limits(regret_df)
#limits.loc[0, ['utility', 'inertia', 'reliability', 'max_P']] = 0

paraxes = parcoords.ParallelAxes(limits)
paraxes.plot(regret_df)
plt.show()

In [ ]:
overall_regret = {}
max_regret = {}
for outcome in dike_model.outcomes:
    policy_column = experiments_reev['policy']
    
    # create a DataFrame with all the relevent information
    # i.e., policy, scenario_id, and scores
    data = pd.DataFrame({outcome.name: outcomes_reev[outcome.name], 
                         "policy":experiments_reev['policy'],
                         "scenario":experiments_reev['scenario']})
    
    # reorient the data by indexing with policy and scenario id
    data = data.pivot(index='scenario', columns='policy')
    
    # flatten the resulting hierarchical index resulting from 
    # pivoting, (might be a nicer solution possible)
    data.columns = data.columns.get_level_values(1)
    
    # we need to control the broadcasting. 
    # max returns a 1d vector across scenario id. By passing
    # np.newaxis we ensure that the shape is the same as the data
    # next we take the absolute value
    #
    # basically we take the difference of the maximum across 
    # the row and the actual values in the row
    #
    outcome_regret = (data.max(axis=1)[:, np.newaxis] - data).abs()
    
    overall_regret[outcome.name] = outcome_regret
    max_regret[outcome.name] = outcome_regret.max()
    

In [ ]:
max_regret = pd.DataFrame(max_regret)

fig, ax = plt.subplots(figsize=(10,20))
sns.heatmap(max_regret/max_regret.max(), cmap='viridis', annot=False, ax=ax)
plt.show()

In [ ]:
colors = sns.color_palette()

data = max_regret

limits = parcoords.get_limits(data)
#limits.loc[0, ['utility', 'inertia', 'reliability', 'max_P']] = 0

paraxes = parcoords.ParallelAxes(limits)
paraxes.plot(data)
plt.show()

In [ ]:
max_regret_filtered = max_regret.sort_values(by=['Expected Number of Deaths']).head(5)
max_regret_filtered

In [ ]:
limits = parcoords.get_limits(max_regret_filtered)
#limits.loc[0, ['utility', 'inertia', 'reliability', 'max_P']] = 0

paraxes = parcoords.ParallelAxes(limits)
paraxes.plot(max_regret_filtered)
plt.show()

In [ ]:
import pandas as pd
from sklearn import preprocessing

x = max_regret_filtered.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
max_regret_filtered_norm = pd.DataFrame(x_scaled, columns=max_regret_filtered.columns)


limits = parcoords.get_limits(max_regret_filtered_norm)
#limits.loc[0, ['utility', 'inertia', 'reliability', 'max_P']] = 0

paraxes = parcoords.ParallelAxes(limits)
paraxes.plot(max_regret_filtered_norm)
plt.show()

In [ ]:
from collections import defaultdict

policy_regret = defaultdict(dict)
for key, value in overall_regret.items():
    for policy in value:
        policy_regret[policy][key] = value[policy]

In [ ]:
# this generates a 2 by 2 axes grid, with a shared X and Y axis
# accross all plots
fig, axes = plt.subplots(ncols=2, nrows=2, figsize=(10,10), 
                         sharey=True, sharex=True)

# to ensure easy iteration over the axes grid, we turn it
# into a list. Because there are four plots, I hard coded
# this. 
axes = [axes[0,0], axes[0,1],        
        axes[1,0],axes[1,1]]

# zip allows us to zip together the list of axes and the list of 
# key value pairs return by items. If we iterate over this
# it returns a tuple of length 2. The first item is the ax
# the second items is the key value pair.
for ax, (policy, regret) in zip(axes, policy_regret.items()):
    data = pd.DataFrame(regret)

    # we need to scale the regret to ensure fair visual
    # comparison. We can do that by divding by the maximum regret
    data = data/max_regret.max(axis=0)
    sns.boxplot(data=data, ax=ax)
    
    # removes top and left hand black outline of axes
    sns.despine()
    
    # ensure we know which policy the figure is for
    ax.set_title(str(policy))
fig.autofmt_xdate()
plt.show()

In [ ]:
#esto sirve para filtrar de outcomes
outcomes_reev_logical = outcomes_reev['Expected Number of Deaths'] <= 0.0001
experiments_reev_of_interest = experiments_reev.loc[outcomes_reev_logical]
# outcomes_reev_of_interest_df = pd.DataFrame({k:v[outcomes_reev_logical] for k,v in outcomes_reev.items()})

In [ ]:
max_rfr_costs = 1.1e9
max_dike_costs = 304e6
outcomes_reev_logical_2 = outcomes_reev_of_interest_df['RfR Investment Costs'] <= max_rfr_costs
outcomes_reev_of_interest_df = pd.DataFrame({k:v[outcomes_reev_logical_2] for k,v in outcomes_reev_of_interest_df.items()})

In [ ]:
outcomes_reev_of_interest_df

In [ ]:
# plot results of domain criterion
limits = parcoords.get_limits(outcomes_reev_of_interest_df)
paraxes = parcoords.ParallelAxes(limits)
paraxes.plot(outcomes_reev_of_interest_df)
plt.rcParams["figure.figsize"] = (10,15)
plt.show()

# End of document

# 1.3 Re-evaluate candidate solutions under uncertainty

In [ ]:
from ema_workbench import Policy

policies = archive.drop([o.name for o in robustnes_functions], axis=1)
policies_to_evaluate = []

for i, policy in policies.iterrows():
    policies_to_evaluate.append(Policy("moro {}".format(i), **policy.to_dict()))

In [ ]:
n_scenarios = 1000
with MultiprocessingEvaluator(model) as evaluator:
    results = evaluator.perform_experiments(n_scenarios,
                                            policies_to_evaluate)

#start = time.time()
#end = time.time()

#print('Processing time:',(end-start)/60,'Minutes')

In [ ]:
from ema_workbench import save_results

save_results(results, 'MORO_reevaluation.tar.gz')

In [ ]:
policies.to_csv('moro polices.csv')

In [ ]:
import pandas as pd
experiments, outcomes = results

overall_robustness = {}
for policy in np.unique(experiments['policy']):
    policy_robustness = {}

    logical = experiments['policy'] == policy
    
    for outcome, values in outcomes.items():
        values = values[logical]
        policy_robustness[outcome] = robustness_funcs[outcome](values)
    overall_robustness[policy] = policy_robustness
overall_robustness = pd.DataFrame.from_dict(overall_robustness).T
overall_robustness

In [ ]:

data = overall_robustness.loc[:, 
                              ['Expected Number of Deaths', 'Expected Annual Damage', 'Total Investment Costs']]
limits = parcoords.get_limits(data)
limits.loc[0, :] = 0
limits.loc[1, :] = 1

paraxes = parcoords.ParallelAxes(limits)
paraxes.plot(data)
plt.show()